In [11]:
!pip install -U scikit-learn scipy matplotlib

     ---------------------------------------- 9.2/9.2 MB 59.2 MB/s eta 0:00:00
     --------------------------------------- 44.0/44.0 MB 81.8 MB/s eta 0:00:00
     ---------------------------------------- 302.0/302.0 kB ? eta 0:00:00



[notice] A new release of pip available: 22.2.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [11]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('./spotify_data.csv')

In [12]:
m = df['popularity'].quantile(0.99)
songs = df.copy().loc[df['popularity'] >= m]
songs = songs.sort_values('popularity', ascending=False)

In [5]:
songs = songs.drop(columns=["Unnamed: 0", "year", "duration_ms"])

In [6]:
songs.head()

,artist_name,track_name,track_id,popularity,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
612503,Miley Cyrus,Flowers,0yLdNVWF3Srea0uzk55zFn,100,pop,0.707,0.681,0,-4.325,1,0.0668,0.0632,0.000005,0.0322,0.646,117.999,4
605178,Bizarrap,"Shakira: Bzrp Music Sessions, Vol. 53",4nrPB8O7Y7wsOCJdgXkthe,96,hip-hop,0.778,0.632,2,-5.600,0,0.0493,0.2740,0.000000,0.0915,0.498,122.104,4
612504,The Weeknd,Die For You - Remix,7oDd86yk8itslrA9HRP2ki,95,pop,0.531,0.525,1,-6.500,0,0.0671,0.2320,0.000000,0.4410,0.502,66.900,4
569184,SZA,Kill Bill,1Qrg8KqiBpW07V7PNxwwwL,94,pop,0.644,0.735,8,-5.747,1,0.0391,0.0521,0.144000,0.1610,0.418,88.980,4
569195,Manuel Turizo,La Bachata,5ww2BF9slyYgNOk37BlC4u,93,pop,0.835,0.679,7,-5.329,0,0.0364,0.5830,0.000002,0.2180,0.850,124.980,4


In [40]:
tempSongsDF = songs.drop(columns=["track_name", "popularity", "track_id", "artist_name"])

In [41]:
onehot_encoded = pd.get_dummies(tempSongsDF['genre'])
# Concatenate the one-hot encoded columns with the original DataFrame
tempSongsDF = pd.concat([tempSongsDF, onehot_encoded], axis=1)
# Drop the original "genre" column if desired
tempSongsDF.drop('genre', axis=1, inplace=True)

In [27]:
#Label Encoding
#This is probably not preferable since numbers imply that some genres are more similar to others.
#Holy fuck label encoding sucks do not use.
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
tempSongsDF['genre_encoded'] = label_encoder.fit_transform(tempSongsDF['genre'])
tempSongsDF.drop('genre', axis=1, inplace=True)

In [42]:
#COSINE SIMILARITY
from sklearn.metrics.pairwise import cosine_similarity

cs = cosine_similarity(tempSongsDF)
print(cs)

[[1.         0.99971563 0.99766922 ... 0.99983453 0.99250213 0.99823476]
 [0.99971563 1.         0.99817242 ... 0.9997369  0.9944643  0.99914898]
 [0.99766922 0.99817242 1.         ... 0.99756233 0.99672979 0.99839984]
 ...
 [0.99983453 0.9997369  0.99756233 ... 1.         0.99291419 0.99860145]
 [0.99250213 0.9944643  0.99672979 ... 0.99291419 1.         0.99718737]
 [0.99823476 0.99914898 0.99839984 ... 0.99860145 0.99718737 1.        ]]


In [5]:
songs_ids = pd.Series(range(songs['track_id'].size), index=songs['track_id'])
songs_ids

track_id
0yLdNVWF3Srea0uzk55zFn        0
4nrPB8O7Y7wsOCJdgXkthe        1
7oDd86yk8itslrA9HRP2ki        2
1Qrg8KqiBpW07V7PNxwwwL        3
5ww2BF9slyYgNOk37BlC4u        4
                          ...  
6SG8lh7fWQ2bahP5WBYePn    12044
5XS0GCCIotaI6XtsYcIKeX    12045
28xjm4FnnGI4Xnds7VoNSl    12046
3HBNf1kBZB3Sn4idl26aJK    12047
4NDpn6yfD5aqafYWFEaJDz    12048
Length: 12049, dtype: int64

In [44]:
def song_recommender(song_id, cs=cs):
    idx = songs_ids[song_id]
    scores = list(enumerate(cs[idx]))
    sorted_scores = sorted(scores, key= lambda x:x[1], reverse=True)[1:11]
    rec_songs_idxs = [i[0] for i in sorted_scores]
    print("\nTop 10 similar songs:\n")
    return songs.iloc[rec_songs_idxs]

In [45]:
def getSongId(song_name, artist_name=""):
    if artist_name == "":
        ids = songs[songs["track_name"] == song_name]['track_id']
        if ids.size > 1:
            raise Exception("Provide artist name, song appears multiple times.")
        else:
            return ids.iloc[0]
    else:
        ids = songs[songs["track_name"] == song_name]
        ids = ids[ids["artist_name"] == artist_name]['track_id']
        return ids.iloc[0]

In [46]:
def song_recommender_name(song_name, artist_name=""):
    return song_recommender(getSongId(song_name, artist_name=artist_name))

In [47]:
my_df = song_recommender_name("Hype Boy")


Top 10 similar songs:



In [48]:
my_df

,artist_name,track_name,track_id,popularity,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
456873,(G)I-DLE,Oh my god,2DmRXiyn03tOqKgEJXlaiJ,68,k-pop,0.794,0.762,4,-3.525,0,0.0392,0.03580,0.000001,0.1910,0.482,109.970,4
564166,SEVENTEEN,HOT,6I2tqFhk8tq69iursYxuxd,75,k-pop,0.765,0.777,5,-4.376,0,0.0399,0.05390,0.000003,0.3380,0.640,111.944,4
401176,NCT DREAM,BOOM,6rcn967QN5JtkPOBDwYIuT,66,k-pop,0.914,0.516,5,-4.178,0,0.0918,0.13100,0.000000,0.0441,0.702,119.969,4
564187,SEULGI,28 Reasons,1dfsPqH09vnzUWEOsN98Ex,70,k-pop,0.766,0.651,5,-3.546,0,0.0952,0.21100,0.000003,0.1440,0.488,121.995,4
344874,BTS,Trivia 轉 : Seesaw,0mZI1NpihIVcho2f9MmqSW,67,k-pop,0.707,0.791,5,-4.483,0,0.0350,0.17900,0.000000,0.1020,0.674,106.985,4
564226,Stray Kids,SUPER BOARD,078fJxqEUz0LvbHSM6AoXJ,65,k-pop,0.751,0.870,4,-2.551,0,0.0713,0.00653,0.000000,0.0770,0.412,103.019,4
247531,BLACKPINK,PLAYING WITH FIRE,7qmvLmX9tyaTiBAVNI6YEn,74,k-pop,0.706,0.756,4,-4.405,0,0.0845,0.03920,0.000000,0.1800,0.676,97.098,4
456884,BTS,Telepathy,6Fnvi5QnVkTskSzeRvvQds,67,k-pop,0.787,0.669,5,-4.333,0,0.0428,0.00782,0.000000,0.1230,0.570,104.994,4
564246,NCT DREAM,Beatbox,0CatzXH85XWyBqqdB6qPMB,69,k-pop,0.857,0.700,4,-4.189,0,0.0740,0.04110,0.000000,0.0334,0.773,93.997,4
288436,BTS,dimple,6pp6xf9zS55GD4nr318XUH,70,k-pop,0.612,0.844,5,-4.694,0,0.0757,0.14000,0.000000,0.2890,0.652,134.872,4


In [51]:
import json
from json import JSONEncoder

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)


numpyData = {"array": cs}
encodedNumpyData = json.dumps(numpyData, cls=NumpyArrayEncoder)

In [ ]:
encodedNumpyData

In [7]:
songs_ids.to_csv('song_ids.csv', index=True)

In [6]:
songs.to_csv('songs.csv', index=True)

In [18]:
exports = songs[["artist_name", "track_name", "track_id", "year", "genre"]]

In [25]:
exports.to_json(path_or_buf="tracks.json", orient="records")